# PySTAC Client の簡単なデモ

2022-11-12 \
[FOSS4G 2022 Japan Online「STAC からはじめる地球観測衛星データの利活用」](https://www.osgeo.jp/events/foss4g-2022/foss4g-2022-japan-online/foss4g-japan-2022-online-core-day#presentation1)

In [1]:
from pystac_client import Client

AWS にオープンデータとしてホストされている [Sentinel-2 Cloud-Optimized GeoTIFFs](https://aws.amazon.com/marketplace/pp/prodview-ykj5gyumkzlme) の STAC API を利用します。

In [2]:
client = Client.open('https://earth-search.aws.element84.com/v0')
client

ID: earth-search
Title: Earth Search
Description: A STAC API of AWS Public Datasets powered by stac-server
stac_api_version: 0.9.0
type: Catalog
ID: sentinel-s2-l2a
Title: Sentinel 2 L2A
"Description: Sentinel-2a and Sentinel-2b imagery, processed to Level 2A (Surface Reflectance)"
Providers: ESA (producer) Sinergise (processor) AWS (host) Element 84 (processor)
stac_extensions: ['https://stac-extensions.github.io/item-assets/v1.0.0/schema.json']
title: Sentinel 2 L2A


`sentinel-s2-l2a-cogs` の Collection の詳細を確認します。

In [3]:
collection = client.get_collection('sentinel-s2-l2a-cogs')
collection

ID: sentinel-s2-l2a-cogs
Title: Sentinel 2 L2A COGs
"Description: Sentinel-2a and Sentinel-2b imagery, processed to Level 2A (Surface Reflectance) and converted to Cloud-Optimized GeoTIFFs"
Providers: ESA (producer) Sinergise (processor) AWS (host) Element 84 (processor)
stac_extensions: ['item-assets']
title: Sentinel 2 L2A COGs
"keywords: ['sentinel', 'earth observation', 'esa']"
"providers: [{'name': 'ESA', 'roles': ['producer'], 'url': 'https://earth.esa.int/web/guest/home'}, {'name': 'Sinergise', 'roles': ['processor'], 'url': 'https://registry.opendata.aws/sentinel-2/'}, {'name': 'AWS', 'roles': ['host'], 'url': 'http://sentinel-pds.s3-website.eu-central-1.amazonaws.com/'}, {'name': 'Element 84', 'roles': ['processor'], 'url': 'https://element84.com'}]"
"summaries: {'platform': ['sentinel-2a', 'sentinel-2b'], 'constellation': ['sentinel-2'], 'instruments': ['msi'], 'gsd': [10], 'view:off_nadir': [0]}"
item-assets
ID: S2A_52NBG_20221111_0_L2A


STAC API - Item Search を使って FOSS4G 2022 Japan の会場がある荒川区を含むデータを検索してみます。

[sankichi92/shikuchoson-boundaries](https://github.com/sankichi92/shikuchoson-boundaries) を使って荒川区（行政区域コード: 13118）の行政区域界の GeoJSON を取得します。

In [4]:
import requests

arakawaku = requests.get('https://shikuchoson-boundaries.sankichi.app/13118.geojson').json()
arakawaku['bbox']

[139.74963749727272,
 35.724611000697166,
 139.81466119355935,
 35.754855171429085]

2022年10月に観測された荒川区を含む `sentinel-s2-l2a-cogs` のデータを検索します。

In [5]:
results = client.search(
    collections=collection,
    bbox=arakawaku['bbox'],
    datetime='2022-10-01/2022-10-31',
)
results.matched()

6

6件のデータがヒットしました。

このうち、もっとも雲が少ない Item を確認します。 \
雲の割合は [Electro-Optical Extension](https://github.com/stac-extensions/eo) により、`eo:cloud_cover` で確認できます。

In [6]:
min_cloud_item = min(results.items(), key=lambda item: item.properties['eo:cloud_cover'])
min_cloud_item

ID: S2A_54SUE_20221029_0_L2A
"Bounding Box: [138.77761489199077, 35.135011161377925, 140.0097096778842, 36.14069288925992]"
Datetime: 2022-10-29 01:37:25+00:00
datetime: 2022-10-29T01:37:25Z
platform: sentinel-2a
constellation: sentinel-2
instruments: ['msi']
gsd: 10
view:off_nadir: 0
proj:epsg: 32654
sentinel:utm_zone: 54


Assets よりサムネイルを表示してみます。

In [7]:
from IPython.display import Image

Image(url=min_cloud_item.assets["thumbnail"].href)

以上、STAC API - Item Search による地球観測衛星データの簡単な検索でした。

もっと複雑な検索も可能で、詳細は各ドキュメントを参照してください。

- [STAC API - Item Search Specification](https://github.com/radiantearth/stac-api-spec/tree/main/item-search)
- [PySTAC Client](https://pystac-client.readthedocs.io/)

また、ここでは検索までしか紹介しませんが、検索で得られたデータを使って様々なことができます。 \
より高度な利用方法については https://stacindex.org/learn のチュートリアルなどを参照してみてください。